## Tipos de Dados

Referencia de Documetação https://spark.apache.org/docs/latest/api/python/reference/index.html

Nesta seção, vamos explorar os principais tipos de dados utilizados no Spark, fundamentais para manipulação, análise e transformação de dados em ambientes distribuídos. Compreender esses tipos é essencial para garantir operações eficientes e corretas ao trabalhar com DataFrames e outras estruturas de dados no Spark.

* **nullable**: isso indica que alguns dos registros nessa coluna podem ter valores nulos.

* **string ou varchar**: Texto.

* **integer ou int**: Numero inteiro.

* **long**: Numero inteiro longo.

* **double**: precisão dupla.

* **float**: precisão simples.

* **decimal**: decimal.

* **timestamp**: Data e hora.

* **date**: Somente a Data.

* **boolean**: booleana (True e False) 0=False e 1=True.

In [0]:
path = "/Volumes/workspace/default/tutorial"
df = spark.read.json(f"{path}/anac/V_OCORRENCIA_AMPLA.json")
display(df)

In [0]:
orders_df = spark.read. \
    format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(f"{path}/bike-store/orders.csv")

display(orders_df)

In [0]:
orders_df.printSchema()

In [0]:
# Ver nomes das colunas 
print(df.columns)

In [0]:
for Loop in df.columns:
    print(Loop)

### Selecionando apenas algumas colunas do DF

Ao trabalhar com DataFrames no Spark, muitas vezes é necessário selecionar apenas um subconjunto de colunas relevantes para a análise ou transformação desejada. Essa seleção permite otimizar o processamento, facilitar a visualização dos dados e preparar o DataFrame para etapas subsequentes do pipeline de dados.

In [0]:
display(df['Classificacao_da_Ocorrência','Danos_a_Aeronave','UF','Municipio','Ilesos_Passageiros'])


In [0]:
# Salvando em um novo Df
Filtrado = df['Classificacao_da_Ocorrência','Danos_a_Aeronave','UF','Municipio','Ilesos_Passageiros']
display(Filtrado)

In [0]:
df.columns

In [0]:
# Criando uma variavel com as colunas (lista)
ColunasSelecionadas = [
    'Categoria_da_Aeronave',
    'Classificacao_da_Ocorrência',
    'Danos_a_Aeronave',
    'Data_da_Ocorrencia',
    'Descricao_do_Tipo',
    'Fase_da_Operacao'
]

display(df[ColunasSelecionadas])

In [0]:
# Salvando em um Df 
ColunasSelecionadas = [
    'Categoria_da_Aeronave',
    'Classificacao_da_Ocorrência',
    'Danos_a_Aeronave',
    'Data_da_Ocorrencia',
    'Descricao_do_Tipo',
    'Fase_da_Operacao'
]

Novo = df[ColunasSelecionadas]
display(Novo)

In [0]:
# Método Select
display(df.select('Danos_a_Aeronave','Data_da_Ocorrencia','Operador', 'UF') )

###Criando novas Colunas

Ao trabalhar com DataFrames no Spark, frequentemente é necessário criar novas colunas a partir de transformações, cálculos ou condições aplicadas sobre os dados existentes. Essa prática é fundamental para enriquecer o conjunto de dados, facilitar análises e preparar informações para etapas posteriores do pipeline de dados. Nesta seção, veremos como adicionar novas colunas utilizando diferentes abordagens no Spark.

In [0]:
teste = df['Classificacao_da_Ocorrência','UF','Municipio']

In [0]:
display(teste)

In [0]:
from pyspark.sql.functions import lit
display(teste.withColumn("TesteTexto",lit(10)))

In [0]:
# Nova coluna com minicipio - UF 
# Lit = Trazer o valor literal para a coluna ex uma coluna com todas as linhas com o valor 10 ou com a palava Brasil

from pyspark.sql.functions import concat, lit
display(teste.withColumn("Municipio_UF", concat(teste.Municipio, lit(" - "), teste["UF"])))

In [0]:
# Salvando em Df
from pyspark.sql.functions import concat, lit
Tratado = teste.withColumn("Municipio_UF", concat(teste.Municipio, lit(" - "),df["UF"]))
display(Tratado)

In [0]:
# Inserindo mais de uma coluna no mesmo comando exemplo uma coluna com país, outra com Municipio em letra miniscula
from pyspark.sql.functions import concat,lit,lower
Tratado2 = teste\
    .withColumn("Municipio_UF", concat(teste.Municipio, lit(" - "), df["UF"])) \
    .withColumn("País", lit("Brasil")) \
    .withColumn("Minisculo", lower(teste.Municipio))

display(Tratado2)

###Criando Coluna Condicional

Ao manipular dados em DataFrames no Spark, muitas vezes é necessário criar colunas cujos valores dependem de condições específicas aplicadas sobre os dados existentes. As colunas condicionais permitem enriquecer o conjunto de dados com informações derivadas de regras de negócio, facilitando análises e tomadas de decisão. Nesta seção, veremos como implementar colunas condicionais de forma eficiente utilizando as funcionalidades do Spark.

In [0]:
Regiao = df['Danos_a_Aeronave', 'UF']
display(Regiao)

In [0]:
# Estado \ Regiao 
"""
'AC': 'Norte',
'AL': 'Nordeste',
'AP': 'Norte',
'AM': 'Norte',
'BA': 'Nordeste',
'CE': 'Nordeste',
'ES': 'Sudeste',
'GO': 'Centro-Oeste',
'MA': 'Nordeste',
'MT': 'Centro-Oeste',
'MS': 'Centro-Oeste',
'MG': 'Sudeste',
'PA': 'Norte',
'PB': 'Nordeste',
'PR': 'Sul',
'PE': 'Nordeste',
'PI': 'Nordeste',
'RJ': 'Sudeste',
'RN': 'Nordeste',
'RS': 'Sul',
'RO': 'Norte',
'RR': 'Norte',
'SC': 'Sul',
'SP': 'Sudeste',
'SE': 'Nordeste',
'TO': 'Norte'
"""


In [0]:
from pyspark.sql.functions import when
Regiao = Regiao \
    .withColumn(
        "Regiao",
        when(df.UF == "MG", "Sudeste") \
        .otherwise("Outra Regiao")
    )

display(Regiao)
#When = quando 
#otherwise = outra Forma (senão)

In [0]:
# Mais de uma condição isin - esta em (Estra dentro de algo).
# Muito usada para verificar se os valores de uma coluna ou expressão estão contidos em um conjunto de valores especificados
from pyspark.sql.functions import when
Regiao = Regiao\
    .withColumn(
        "Regiao",
        when(df.UF.isin ("MG","SP","RJ","ES"), "Sudeste") \
        .otherwise("Outra Regiao")
    )

display(Regiao)

In [0]:
# Fazendo o mesmo Para regiao Sul [PR, RS, SC]
from pyspark.sql.functions import when
Regiao = Regiao \
    .withColumn(
        "Regiao",
        when(df.UF.isin ("MG","SP","RJ","ES"), "Sudeste") \
        .when(df.UF.isin ("PR","RS","SC"), "Sul" ) \
        .otherwise("Outra Regiao")
    )

display(Regiao)

### Coluna Condicional de valor

Ao trabalhar com DataFrames no Spark, é comum precisar criar colunas cujos valores são definidos a partir de condições específicas. Essas colunas condicionais permitem atribuir diferentes valores conforme regras de negócio ou critérios de análise, enriquecendo o conjunto de dados e facilitando a tomada de decisões. Nesta seção, veremos como criar colunas condicionais utilizando funções do Spark, tornando o processo flexível e eficiente para diferentes cenários.

In [0]:
#Nova coluna com descrição Status de Ordem 
from pyspark.sql.functions import when

orders_df = orders_df \
    .withColumn("DescStatus",
        when(orders_df.order_status == 1 , "Pendente") \
        .otherwise("Sem Status")
    )

display(orders_df)

In [0]:
# inserindo Varias condições (Simulando Status de produção)
# 1 = Pendente
# 2 = Em produção
# 3 = Em rota de entrega
# 4 = Finalizado
from pyspark.sql.functions import when

orders_df = orders_df.withColumn(
    "DescStatus",
    when (orders_df.order_status == 1 , "Pendente") \
    .when (orders_df.order_status == 2 , "Em produção") \
    .when (orders_df.order_status == 3 , "Em rota de entrega") \
    .otherwise("Finalizado")
)

display(orders_df)

In [0]:
order_items = spark.read. \
    format("csv")\
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(f"{path}/bike-store/order_items.csv")

display(order_items)

In [0]:
# caso for um valor especifico traga uma informação 
from pyspark.sql.functions import when

order_items = order_items.withColumn(
    "DescStatus",
    when(order_items.list_price == 599.99 , "valor procurado") \
    .otherwise("Não")
)

display(order_items)

In [0]:
#Filtro do Valor procurado
display(order_items.filter(order_items.DescStatus == "valor procurado"))

In [0]:
# Sumilado Com varios Valores
# 0 a 500    = Barato
# 501 a 1500 = Médio
# > 1500     = Caro
from pyspark.sql.functions import when

Taxa = order_items.withColumn(
    "TaxaCliente",
    when((order_items.list_price >= 0) & (order_items.list_price <= 500), "Barato") \
    .when((order_items.list_price >= 501) & (order_items.list_price <= 1500), "Médio") \
    .when(order_items.list_price > 1500, "Caro") \
    .otherwise("Caro")
)

display(Taxa)

In [0]:
#Fazendo Filtro para validar 
display(Taxa.filter(Taxa.TaxaCliente == "Caro"))

### Comando Sql para Selecionar Colunas e Criar Novas

Ao utilizar comandos SQL no Spark, é possível selecionar colunas específicas de um DataFrame e criar novas colunas diretamente na consulta. Essa abordagem facilita a manipulação e transformação dos dados, permitindo aplicar funções, expressões e condições para gerar colunas derivadas conforme a necessidade da análise.

In [0]:
# Inserindo em uma tabela temporaria para 
df.createOrReplaceTempView("anac")

In [0]:
print(df.columns)

In [0]:
# Selecionando algumas colunas
Novodf = spark.sql("""
select
  Danos_a_Aeronave,
  Municipio,
  UF
from anac
""")

display(Novodf)

In [0]:
# Coluna condicional  "MG","SP","RJ","ES" = "Sudeste"
Novodf = spark.sql("""
select
   Danos_a_Aeronave,
   Municipio,
   UF,
   Case
      when UF in ('MG','SP','RJ','ES') then 'Sudeste'
      else 'Sem Regiao'
      end as Regiao
from anac
""")

display(Novodf)

In [0]:
# Coluna condicional  "MG","SP","RJ","ES" =  "Sudeste" e 'PR',RS,SC = Sul 

Novodf = spark.sql("""
select
   Danos_a_Aeronave,
   Municipio,
   UF,
   Case
      when UF in ('MG','SP','RJ','ES') then 'Sudeste'
      when UF in ('PR','RS','SC') then 'Sul'
      else 'Sem Regiao'
      end as Regiao
from anac 
""")

display(Novodf)

In [0]:
display(Novodf.filter(Novodf.Regiao == 'Sudeste'))

In [0]:
# Inserindo em tabela temporaria para rodar SQL
order_items.createOrReplaceTempView("Vendas")

In [0]:
# Consulta Para identificar tipo de cliente a partir da quantidade de compra 
Testevendas = spark.sql("""
select *, Case 
    when quantity = 1 then 'Mâo de Vaca'
    else 'Top'
    end TipoCliente
from Vendas
""")

display(Testevendas)

In [0]:
# Classificação por faixa de Preço
# 0 a 500    = Barato
# 501 a 1500 = Médio
# > 1500     = Caro

Testevendas = spark.sql("""
select *, case 
    when list_price between   0 and 500 then 'Barato'
    when list_price between 501 and 1500 then 'Médio'
    when list_price > 1500 then 'Médio'
    else 'Sem Perfil' 
    end as PerfilProduto
from Vendas
""")

display(Testevendas)